<a href="https://colab.research.google.com/github/RoyDibs/Rebars-and-debris-Classiffication/blob/master/CNN_RebarsClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and augmenting images
from skimage.io import imread
from skimage.transform import rotate, resize

# for splitting train-test set and evaluating the model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# PyTorch libraries 
import torch
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam

!unzip images2.zip

Archive:  images2.zip
   creating: images2/
  inflating: images2/0.png           
  inflating: images2/1.png           
  inflating: images2/10.png          
  inflating: images2/100.png         
  inflating: images2/101.png         
  inflating: images2/102.png         
  inflating: images2/103.png         
  inflating: images2/104.png         
  inflating: images2/105.png         
  inflating: images2/106.png         
  inflating: images2/107.png         
  inflating: images2/108.png         
  inflating: images2/109.png         
  inflating: images2/11.png          
  inflating: images2/110.png         
  inflating: images2/111.png         
  inflating: images2/112.png         
  inflating: images2/113.png         
  inflating: images2/114.png         
  inflating: images2/115.png         
  inflating: images2/116.png         
  inflating: images2/117.png         
  inflating: images2/118.png         
  inflating: images2/119.png         
  inflating: images2/12.png          
  infl

In [ ]:
train = pd.read_csv('/content/rebars2.csv')

# loading training images
train_img = []
train_img_values = []
for img_name in train['image_names']:
    image_path = '/content/images2/' + img_name
    img = imread(image_path)
    img = img/255
    img = resize(img, output_shape=(3, 224, 224),
                 mode='constant', anti_aliasing=True)
    train_img.append(img)


train_x = np.array(train_img)

# defining the labels
train_y = train['rebars_or_not'].values

train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=13, stratify=train_y)
print("Number of images (Before Image Augmentation) in Training set : ",train_x.shape[0],"  Number of images in Testing set : ",test_x.shape[0])

Number of images (Before Image Augmentation) in Training set :  160   Number of images in Testing set :  41


In [ ]:
# Image Augmentation
final_train_x = []
final_train_y = []

#train_x.shape[0]
for i in range(train_x.shape[0]):
    final_train_x.append(train_x[i])
    final_train_x.append(rotate(train_x[i], angle=45, mode = 'wrap'))
    final_train_x.append(np.fliplr(train_x[i]))
    for j in range(3):
      final_train_y.append(train_y[i])

train_x = np.array(final_train_x)
train_y = np.array(final_train_y)
print("Number of images (After Image Augmentation) in Training set : ",train_x.shape[0],"  Number of images in Testing set : ",test_x.shape[0])

Number of images (After Image Augmentation) in Training set :  1194   Number of images in Testing set :  100


In [ ]:
# train_x = train_x.numpy()
# train_y = train_y.numpy()
# test_x = test_x.numpy()
# test_y = test_y.numpy()
# converting training images and its labels into torch format
train_x = train_x.reshape(train_x.shape[0], 3, 224, 224) #1956 is the number of training images
train_x  = torch.from_numpy(train_x)
train_x = train_x.float()
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)


# converting test images and its labels into torch format
test_x = test_x.reshape(test_x.shape[0], 3, 224, 224) #164 is the number of test images
test_x  = torch.from_numpy(test_x)
test_x = test_x.float()
test_y = test_y.astype(int)
test_y = torch.from_numpy(test_y)

In [ ]:
# Define the model
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(p=0.25),
            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(p=0.25),
            Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(p=0.25),
            Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(p=0.25),
        )

        self.linear_layers = Sequential(
            Linear(128 * 14 * 14, 512),
            ReLU(inplace=True),
            Dropout(),
            Linear(512, 256),
            ReLU(inplace=True),
            Dropout(),
            Linear(256,10),
            ReLU(inplace=True),
            Dropout(),
            Linear(10,2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
model = Net()
optimizer = Adam(model.parameters(), lr=0.000075)
criterion = CrossEntropyLoss()

# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [ ]:
# batch size of the model
batch_size = 32

# number of epochs to train the model
n_epochs = 50

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
    permutation = torch.randperm(train_x.size()[0])
    training_loss = []
    for i in range(0,train_x.size()[0], batch_size):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step() 
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

epoch: 	 1 	 training loss: 	 0.49921850389555883
epoch: 	 2 	 training loss: 	 0.39909473845833227
epoch: 	 3 	 training loss: 	 0.36141975537726756
epoch: 	 4 	 training loss: 	 0.33978884784798874
epoch: 	 5 	 training loss: 	 0.3245070313936786
epoch: 	 6 	 training loss: 	 0.3105532381879656
epoch: 	 7 	 training loss: 	 0.3094852525544794
epoch: 	 8 	 training loss: 	 0.29419052718501343
epoch: 	 9 	 training loss: 	 0.2969778023268047
epoch: 	 10 	 training loss: 	 0.30031209164544154
epoch: 	 11 	 training loss: 	 0.29057781417903145
epoch: 	 12 	 training loss: 	 0.288523184625726
epoch: 	 13 	 training loss: 	 0.284906195966821
epoch: 	 14 	 training loss: 	 0.2824834295009312
epoch: 	 15 	 training loss: 	 0.27654242515563965
epoch: 	 16 	 training loss: 	 0.2678015465406995
epoch: 	 17 	 training loss: 	 0.2675497976965026
epoch: 	 18 	 training loss: 	 0.275429762115604
epoch: 	 19 	 training loss: 	 0.2682835818513444
epoch: 	 20 	 training loss: 	 0.2705781346088962
epoc

In [ ]:
# batch size of the model
batch_size = 64
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in range(0,train_x.size()[0], batch_size):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i].cpu(),prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))

training accuracy: 	 0.9489739974937343


In [ ]:
# checking the performance on validation set
torch.manual_seed(0)
# batch size of the model
batch_size = 64
# prediction for test set
prediction = []
target = []
permutation = torch.randperm(test_x.size()[0])
for i in range(0,test_x.size()[0], batch_size):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = test_x[indices], test_y[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# Test accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i].cpu(),prediction[i]))
    
print('Test accuracy: \t', np.average(accuracy))
torch.save(model, 'CNN-model.pth')
     

Test accuracy: 	 0.8958333333333333


In [ ]:
model = torch.load('CNN-model.pth')
image_path = '/content/2.png' #Specify the image path
img = imread(image_path)
img = img/255
img = resize(img, output_shape=(3, 224, 224),
               mode='constant', anti_aliasing=True)
img = img.astype('float32')
img = np.array(img)

img = img.reshape(1, 3, 224, 224) 
img = torch.from_numpy(img)
img = img.float()

output = model(img.cuda())
softmax = torch.exp(output).cpu()
prob = list(softmax.detach().numpy())
predictions = np.argmax(prob, axis=1)
print(predictions)
if predictions == 0:
  print('Image contains debris')
else:
  print('Image contains rebars')


[1]
Image contains rebars
